In [1]:
import urllib.parse
from pathlib import Path

import psycopg
from label_studio_sdk.client import LabelStudio
from label_studio_sdk.projects.client_ext import ProjectExt
from label_studio_sdk.types import BaseUser
from psycopg.rows import dict_row
from tqdm.auto import tqdm

from fishsense_data_processing_spider.backend import (get_file_checksum,
                                                      get_project_export)
from fishsense_data_processing_spider.config import PG_CONN_STR, settings

In [2]:
data_root = Path('./mnt/REEF/data')

In [3]:
export = get_project_export(
    project_id=19,
    label_studio_api_key='8385ef396487bbdc375acf6723c81e90c1afbf89',
    label_studio_host='labeler.e4e.ucsd.edu'
)

In [12]:
images = {task['id']: Path(urllib.parse.urlparse(task['data']['img']).path).stem for task in export}
images

{141418: '0053f52c583e9cfe998aef64cf26c3bd',
 141420: '008debc5fb1c721d6f1c2cc8da403c5c',
 141421: '00ad982a13a88f023679458728a9f292',
 141423: '00ca1ab7beae1ecf953e5bdd7f93f078',
 141424: '00f22ba8e6cee3538f23d3e643b95e4a',
 141426: '01b1bcd8e0330bf7bec88f164f09b7f1',
 141431: '02a16d14f0eeb11ba10fabf135930e75',
 141434: '039f017bca69a18ee0ced1a203a32622',
 141439: '044deb7be6b5b0c77c2ef84876578a5e',
 141445: '05c1143561e7246399a2745bf2e49b27',
 141449: '06696adb1e825f6519a23843d5022284',
 141450: '06735ef603628ece822277e60096ed4e',
 141451: '0712381a19ae1141893d02a0bc26fbe4',
 141454: '077041be7c18fee75875f6bc1b038655',
 141455: '077410fd9dd0d31d65c50609c2b7b78e',
 141457: '0890209f810c4947940387354631c0ef',
 141458: '08ab492272183e60d71d4637e76dac0d',
 141460: '08cb6dff8d7340292d0b734f8742803f',
 141461: '08fdb967385d594d223a53e173108eab',
 141462: '090030382d9bc4caf1bf8e35b3f29e72',
 141463: '0a19d4b48b16a45907a64f37577b1afd',
 141465: '0ac7c4abb5d5c6d739b7177871505d68',
 141467: '

In [13]:
bad_cksums = []
with psycopg.connect(PG_CONN_STR, row_factory=dict_row) as con, con.cursor() as cur:
    for task_id, cksum in tqdm(images.items()):
        cur.execute('SELECT path FROM images WHERE image_md5 = %(cksum)s;', {'cksum': cksum})
        result = cur.fetchall()
        if len(result) == 0:
            bad_cksums.append(cksum)
            continue
        cur.execute(
            'UPDATE images SET headtail_task_id = %(task_id)s WHERE image_md5 = %(cksum)s;',
            {
                'task_id': task_id,
                'cksum': cksum
            })

  0%|          | 0/2019 [00:00<?, ?it/s]

In [14]:
bad_cksums

[]